In [1]:
import requests
import pandas as pd
from datetime import datetime

# Constants
LAT, LON = 28.98, 77.71  # Meerut, UP
START_DATE = "2015-01-01"
END_DATE = "2024-12-31"
NASA_POWER_API = "https://power.larc.nasa.gov/api/temporal/daily/point"

def fetch_weather_data(lat, lon, start, end):
    params = {
        "parameters": "T2M,PRECTOTCORR,WS2M,ALLSKY_SFC_SW_DWN",
        "community": "AG",
        "longitude": lon,
        "latitude": lat,
        "start": start.replace("-", ""),
        "end": end.replace("-", ""),
        "format": "JSON"
    }
    response = requests.get(NASA_POWER_API, params=params)
    data = response.json()["properties"]["parameter"]
    df = pd.DataFrame({
        "date": data["T2M"].keys(),
        "temperature_C": data["T2M"].values(),
        "precip_mm": data["PRECTOTCORR"].values(),
        "wind_m_s": data["WS2M"].values(),
        "solar_rad_MJ_m2": data["ALLSKY_SFC_SW_DWN"].values(),
    })
    df["date"] = pd.to_datetime(df["date"])
    return df

def simulate_irrigation(df):
    df["crop"] = "None"
    df["soil_moisture"] = 30.0
    df["irrigated"] = 0
    df["electricity_kWh"] = 0.0

    for i in range(len(df)):
        month = df.loc[i, "date"].month

        if 6 <= month <= 10:
            df.loc[i, "crop"] = "Rice"
        elif month >= 11 or month <= 3:
            df.loc[i, "crop"] = "Wheat"

        prev_moisture = df.loc[i-1, "soil_moisture"] if i > 0 else 30.0
        evap = 0.3 + 0.05 * df.loc[i, "temperature_C"]
        new_moisture = max(0, min(100, prev_moisture + df.loc[i, "precip_mm"] - evap))
        df.loc[i, "soil_moisture"] = new_moisture

        if df.loc[i, "crop"] != "None" and new_moisture < 25:
            df.loc[i, "irrigated"] = 1
            df.loc[i, "electricity_kWh"] = 4.5  # average value
            df.loc[i, "soil_moisture"] += 10  # from irrigation

    return df

# Run
df_weather = fetch_weather_data(LAT, LON, START_DATE, END_DATE)
df_final = simulate_irrigation(df_weather)
df_final.to_csv("Western_UP_Irrigation_Simulation.csv", index=False)
print("✅ Data saved to 'Western_UP_Irrigation_Simulation.csv'")

✅ Data saved to 'Western_UP_Irrigation_Simulation.csv'


In [2]:
df = pd.read_csv('Western_UP_Irrigation_Simulation.csv')

In [3]:
df.head()

,date,temperature_C,precip_mm,wind_m_s,solar_rad_MJ_m2,crop,soil_moisture,irrigated,electricity_kWh
0,2015-01-01,15.54,0.00,1.14,7.75,Wheat,28.923,0,0.0
1,2015-01-02,16.40,4.10,1.06,2.43,Wheat,31.903,0,0.0
2,2015-01-03,14.30,11.06,1.00,4.18,Wheat,41.948,0,0.0
3,2015-01-04,13.26,0.04,2.46,7.20,Wheat,41.025,0,0.0
4,2015-01-05,12.96,0.03,4.19,12.95,Wheat,40.107,0,0.0


In [4]:
import pandas as pd
import numpy as np

# Define coordinates (can be used later for real data fetching)
districts = {
    "Meerut": (28.98, 77.71),
    "Muzaffarnagar": (29.47, 77.70),
    "Saharanpur": (29.96, 77.55),
    "Baghpat": (28.94, 77.23),
    "Bulandshahr": (28.40, 77.85)
}

# Generate mock data
date_range = pd.date_range(start="2015-01-01", end="2024-12-31", freq="D")
data = []

for district in districts:
    for date in date_range:
        month = date.month

        # Random weather patterns
        temp = np.random.normal(25 + 10*np.cos((month-6)*np.pi/6), 2)
        precip = np.random.poisson(5 if 6 <= month <= 9 else 1)
        wind = np.random.uniform(1, 3)
        solar = np.random.uniform(15, 25)

        # Crop logic
        if 6 <= month <= 10:
            crop = "Rice"
        elif month >= 11 or month <= 3:
            crop = "Wheat"
        else:
            crop = "None"

        # Soil moisture & irrigation
        sm = np.random.uniform(15, 50)
        irrigated = int(crop != "None" and sm < 25)
        electricity_kWh = 4.5 if irrigated else 0

        data.append({
            "district": district,
            "date": date,
            "temperature_C": round(temp, 1),
            "precip_mm": round(precip, 2),
            "wind_m_s": round(wind, 2),
            "solar_rad_MJ_m2": round(solar, 2),
            "crop": crop,
            "soil_moisture": round(sm, 1),
            "irrigated": irrigated,
            "electricity_kWh": electricity_kWh
        })

df = pd.DataFrame(data)
df.to_csv("Western_UP_5_Districts_Irrigation_Data.csv", index=False)
print("✅ Data saved to Western_UP_5_Districts_Irrigation_Data.csv")


✅ Data saved to Western_UP_5_Districts_Irrigation_Data.csv


In [5]:
df1 = pd.read_csv("Western_UP_5_Districts_Irrigation_Data.csv")

In [6]:
df1.head()

,district,date,temperature_C,precip_mm,wind_m_s,solar_rad_MJ_m2,crop,soil_moisture,irrigated,electricity_kWh
0,Meerut,2015-01-01,16.6,1,1.07,17.38,Wheat,34.1,0,0.0
1,Meerut,2015-01-02,14.9,1,2.14,15.96,Wheat,30.8,0,0.0
2,Meerut,2015-01-03,14.1,0,2.33,19.37,Wheat,21.1,1,4.5
3,Meerut,2015-01-04,17.3,2,1.16,23.18,Wheat,34.0,0,0.0
4,Meerut,2015-01-05,19.9,2,1.51,22.66,Wheat,33.8,0,0.0
